##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Explore vision capabilities with the Gemini API

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs/vision"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/vision.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/vision.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

The Gemini API is able to process images and videos, enabling a multitude of
 exciting developer use cases. Some of Gemini's vision capabilities include
 the ability to:

*   Caption and answer questions about images
*   Transcribe and reason over PDFs, including long documents up to 2 million token context window
*   Describe, segment, and extract information from videos,
including both visual frames and audio, up to 90 minutes long
*   Detect objects in an image and return bounding box coordinates for them

This tutorial demonstrates some possible ways to prompt the Gemini API with
images and video input, provides code examples,
and outlines prompting best practices with multimodal vision capabilities.
All output is text-only.

## Setup

Before you use the File API, you need to install the Gemini API SDK package and configure an API key. This section describes how to complete these setup steps.

### Install the Python SDK and import packages

The Python SDK for the Gemini API is contained in the [google-generativeai](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip.

In [1]:
!pip install -q -U google-generativeai

Import the necessary packages.

In [2]:
import google.generativeai as genai
from IPython.display import Markdown

### Set up your API key

The File API uses API keys for authentication and access. Uploaded files are associated with the project linked to the API key. Unlike other Gemini APIs that use API keys, your API key also grants access to data you've uploaded to the File API, so take extra care in keeping your API key secure. For more on keeping your keys
secure, see [Best practices for using API
keys](https://support.google.com/googleapi/answer/6310037).

Store your API key in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or are unfamiliar with Colab Secrets, refer to the [Authentication quickstart](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb).

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Prompting with images

In this tutorial, you will upload images using the File API or as inline data and generate content based on those images.

### Technical details (images)
Gemini 1.5 Pro and Flash support a maximum of 3,600 image files.

Images must be in one of the following image data [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):

-   PNG - `image/png`
-   JPEG - `image/jpeg`
-   WEBP - `image/webp`
-   HEIC - `image/heic`
-   HEIF - `image/heif`

Each image is equivalent to 258 tokens.

While there are no specific limits to the number of pixels in an image besides the model’s context window, larger images are scaled down to a maximum resolution of 3072x3072 while preserving their original aspect ratio, while smaller images are scaled up to 768x768 pixels. There is no cost reduction for images at lower sizes, other than bandwidth, or performance improvement for images at higher resolution.

For best results:

*   Rotate images to the correct orientation before uploading.
*   Avoid blurry images.
*   If using a single image, place the text prompt after the image.

## Image input

For total image payload size less than 20MB, it's recommended to either upload
base64 encoded images or directly upload locally stored image files.

### Base64 encoded images

You can upload public image URLs by encoding them as Base64 payloads.
You can use the httpx library to fetch the image URLs.
The following code example shows how to do this:

In [4]:
import httpx
import base64

# Retrieve an image
image_path = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg/2560px-Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg"
image = httpx.get(image_path)

# Choose a Gemini model
model = genai.GenerativeModel(model_name="gemini-1.5-pro")

# Create a prompt
prompt = "Caption this image."
response = model.generate_content(
    [
        {
            "mime_type": "image/jpeg",
            "data": base64.b64encode(image.content).decode("utf-8"),
        },
        prompt,
    ]
)

Markdown(">" + response.text)

>A panoramic view of London showcases iconic landmarks under a cloudy sky.  The London Eye Ferris wheel is prominent on the left, while the Houses of Parliament and Big Ben stand majestically in the center. The Shard skyscraper peeks out from behind them. The foreground features a mix of buildings and green spaces, with a street winding its way through.  The overcast sky adds a dramatic feel to the cityscape.

### Multiple images

To prompt with multiple images in Base64 encoded format, you can do the
following:

In [6]:
import httpx
import base64

# Retrieve two images
image_path_1 = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg/2560px-Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg"
image_path_2 = "https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg"

image_1 = httpx.get(image_path_1)
image_2 = httpx.get(image_path_2)

# Create a prompt
prompt = "Generate what insights can be taken to make it a movie scene from image 2"

response = model.generate_content([
{'mime_type':'image/jpeg', 'data': base64.b64encode(image_1.content).decode('utf-8')},
{'mime_type':'image/jpeg', 'data': base64.b64encode(image_2.content).decode('utf-8')}, prompt])

Markdown(response.text)

The sketch of the jetpack backpack suggests a movie scene with several possibilities, all centering around the contrast between its mundane appearance and its extraordinary capability:

**Scene Ideas:**

* **Unassuming Hero:**  A seemingly ordinary person (student, office worker, tourist) is late for something important (exam, meeting, flight).  Frustrated by traffic or a slow tube train, they discreetly activate the backpack's boosters, soaring over the London skyline (as seen in the photograph).  The shot could mimic the photograph's perspective, suddenly revealing the jetpack's fiery exhaust. This establishes the character and their secret.
* **Chase Scene:** The protagonist is being pursued through the crowded streets of London. They try blending in but are eventually cornered.  Just when it seems hopeless, the boosters ignite, allowing them to escape spectacularly, perhaps even using the London Eye or other landmarks as obstacles/launching points.
* **Comic Mishap:** Someone tests the jetpack for the first time in a confined space, causing chaos and destruction with the powerful steam bursts. The lightweight nature of the backpack might also play a role – perhaps it easily gets snagged on something, leading to an uncontrolled flight.
* **Stealth Mission:** The backpack's "normal" appearance allows the user to infiltrate a secure location. Once inside, they use the jetpack for a quick escape or to reach a previously inaccessible area, surprising their adversaries.
* **Dystopian Future:**  In a future where personal flight is restricted, the jetpack backpack becomes a symbol of rebellion.  The scene could show a black market exchange of the backpack, or a character using it to flee a totalitarian regime.

**Elements to consider based on the sketch:**

* **Short Battery Life:** The 15-minute battery life creates dramatic tension. The user has limited time to accomplish their goal, adding a ticking clock element.
* **Steam-Powered/Green Energy:** The steam could be visually interesting, creating a unique visual contrast to the typical fiery exhaust of jetpacks.  The "green" aspect could also be a plot point – perhaps the user is an environmentally conscious hero.
* **Retractable Boosters:** The retractability adds an element of surprise.  The backpack could appear ordinary until suddenly transforming.
* **Fits 18" Laptop:** This suggests a modern setting and potentially ties the user to a specific profession (programmer, designer, etc.).


By combining the iconic London setting with the unique features of the jetpack, the scene can be both exciting and memorable.


### Upload one or more locally stored image files

Alternatively, you can upload one or more locally stored image files..

You can download and use our drawings of [piranha-infested waters](https://storage.googleapis.com/generativeai-downloads/images/piranha.jpg) and a [firefighter with a cat](https://storage.googleapis.com/generativeai-downloads/images/firefighter.jpg). First, save these files to your local directory.

Then click **Files** on the left sidebar. For each file, click the **Upload** button, then navigate to that file's location and upload it:

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">

When the combination of files and system instructions that you intend to send is larger than 20 MB in size, use the File API to upload those files. Smaller files can instead be called locally from the Gemini API:


In [36]:
import PIL.Image

sample_file_2 = PIL.Image.open('piranha.jpg')
sample_file_3 = PIL.Image.open('firefighter.jpg')

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import google.generativeai as genai

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Create a prompt.
prompt = "Write an advertising jingle based on the items in both images."

response = model.generate_content([sample_file_2, sample_file_3, prompt])

Markdown(response.text)

(Upbeat, slightly quirky music)

Stuck in a rut?  Can't get home?
Shark in the water, feelin' alone?
Need a rescue, fast and free?
Problem Solver Pro's the company!

Cat in a tree, a firefighter's plight?
We'll get 'em down, day or night!
Big or small, near or far,
Problem Solver Pro's your guiding star!

So call us now, don't delay!
Problem Solver Pro will save the day! 
(Sound of a cheering crowd)
Problem Solver Pro!  Hooray!


Note that these inline data calls don't include many of the features available
through the File API, such as getting file metadata,
[listing](https://ai.google.dev/gemini-api/docs/vision?lang=python#list-files),
or [deleting files](https://ai.google.dev/gemini-api/docs/vision?lang=python#delete-files).

### Large image payloads

#### Upload an image file using the File API

When the combination of files and system instructions that you intend to send is larger than 20 MB in size, use the File API to upload those files.

**NOTE**: The File API lets you store up to 20 GB of files per project, with a per-file maximum size of 2 GB. Files are stored for 48 hours. They can be accessed in that period with your API key, but cannot be downloaded from the API. It is available at no cost in all regions where the Gemini API is available.

Upload the image using [`media.upload`](https://ai.google.dev/api/rest/v1beta/media/upload) and print the URI, which is used as a reference in Gemini API calls.

In [39]:
!curl -o jetpack.jpg https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  349k  100  349k    0     0  2339k      0 --:--:-- --:--:-- --:--:-- 2343k


In [40]:
# Upload the file and print a confirmation.
sample_file = genai.upload_file(path="jetpack.jpg",
                            display_name="Jetpack drawing")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")

Uploaded file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/ia916xpqpyp


The `response` shows that the File API stored the specified `display_name` for the uploaded file and a `uri` to reference the file in Gemini API calls. Use `response` to track how uploaded files are mapped to URIs.

Depending on your use case, you can also store the URIs in structures such as a `dict` or a database.

#### Verify image file upload and get metadata

You can verify the API successfully stored the uploaded file and get its metadata by calling [`files.get`](https://ai.google.dev/api/rest/v1beta/files/get) through the SDK. Only the `name` (and by extension, the `uri`) are unique. Use `display_name` to identify files only if you manage uniqueness yourself.

In [41]:
file = genai.get_file(name=sample_file.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")

Retrieved file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/ia916xpqpyp


Depending on your use case, you can store the URIs in structures, such as a `dict` or a database.

#### Prompt with the uploaded image and text

After uploading the file, you can make GenerateContent requests that reference the File API URI. Select the generative model and provide it with a text prompt and the uploaded image.

In [42]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Prompt the model with text and the previously uploaded image.
response = model.generate_content([sample_file, "Describe how this product might be manufactured."])

Markdown(response.text)

Manufacturing a jetpack backpack like this would be complex, requiring expertise in several areas:

1. **Backpack Construction:**  Traditional backpack manufacturing techniques would be employed for the main body, using fabrics like nylon or canvas, potentially incorporating reinforced stitching and padding.

2. **Jetpack Engine Fabrication:** This is the most challenging part.
    * **Miniaturized Steam Generator:**  Creating a small, lightweight, yet powerful steam generator would be crucial. This could involve advanced materials like titanium or high-strength alloys for the boiler and intricate pipework for water circulation and steam delivery. Precision machining and welding would be essential.
    * **Nozzles:**  Retractable, directional nozzles would need to be designed and manufactured from heat-resistant materials.  Precise engineering and testing would ensure they can handle the steam pressure and temperature.
    * **Fuel Storage:** If it's steam-powered, a water tank needs to be integrated into the design, potentially with a quick-refill system.

3. **Electronics Integration:**
    * **USB-C Charging System:**  Standard USB-C charging circuitry would need to be incorporated for the steam generator.
    * **Battery Management System (BMS):**  A BMS would be necessary to safely manage the battery, control charging, and prevent overcharging/discharging.
    * **Control System:**  A small microcontroller or similar system would manage the jetpack's operation, possibly with user input for thrust control.

4. **Assembly and Testing:**
    * **Precision Assembly:**  The various components would be carefully assembled, requiring specialized tools and skilled labor.
    * **Rigorous Testing:** Extensive testing would be essential to ensure safety and functionality. This would include pressure tests for the steam system, thrust tests for the engine, flight stability tests, and battery life tests.
    * **Quality Control:**  Strict quality control measures throughout the manufacturing process would ensure a reliable and safe product.

5. **Materials Selection:**
    * **Lightweight and Durable Materials:**  Materials like carbon fiber composites, titanium alloys, and high-strength polymers would be crucial to minimize weight while maintaining structural integrity.
    * **Heat-Resistant Materials:**  Materials capable of withstanding high temperatures generated by the steam would be necessary for components like the boiler, nozzles, and surrounding areas.


This jetpack backpack would likely be manufactured in a specialized facility with advanced manufacturing capabilities, including CNC machining, 3D printing, and robotic assembly, with highly skilled engineers and technicians overseeing the process.  Given the complexity and safety implications, certification and regulatory approval would also be significant aspects of the manufacturing process.


## Capabilties

This section outlines specific vision capabilities of the Gemini model, including object detection and bounding box coordinates.

### Get bounding boxes

Gemini models are trained to return bounding box coordinates as relative widths or heights in the range of [0, 1]. These values are then scaled by 1000 and converted to integers. Effectively, the coordinates represent the bounding box on a 1000x1000 pixel version of the image. Therefore, you'll need to convert these coordinates back to the dimensions of your original image to accurately map the bounding boxes.

In [43]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Create a prompt to detect bounding boxes.
prompt = "Return a bounding box for each of the objects in this image in [ymin, xmin, ymax, xmax] format."
response = model.generate_content([sample_file_2, prompt])

Markdown(response.text)

- [370, 244, 658, 327]: stick figure
- [613, 489, 710, 597]: shark
- [468, 738, 557, 942]: sign
- [467, 736, 875, 942]: sign post
- [626, 162, 941, 937]: chasm

The model returns bounding box coordinates in the format
`[ymin, xmin, ymax, xmax]`. To convert these normalized coordinates
to the pixel coordinates of your original image, follow these steps:

1.    Divide each output coordinate by 1000.
1.    Multiply the x-coordinates by the original image width.
1.    Multiply the y-coordinates by the original image height.

To explore more detailed examples of generating bounding box coordinates and
visualizing them on images, review our [Object Detection cookbook example](https://github.com/google-gemini/cookbook/blob/main/examples/Object_detection.ipynb).

## Prompting with video

In this tutorial, you will upload a video using the File API and generate content based on those images.

### Technical details (video)

Gemini 1.5 Pro and Flash support up to approximately an hour of video data.

Video must be in one of the following video format [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):
  -   `video/mp4`
  -   `video/mpeg`
  -   `video/mov`
  -   `video/avi`
  -   `video/x-flv`
  -   `video/mpg`
  -   `video/webm`
  -   `video/wmv`
  -   `video/3gpp`

The File API service currently extracts image frames from videos at 1 frame per second (FPS) and audio at 1Kbps, single channel, adding timestamps every second. These rates are subject to change in the future for improvements in inference.

**NOTE:** The finer details of fast action sequences may be lost at the 1 FPS frame sampling rate. Consider slowing down high-speed clips for improved inference quality.

Individual frames are 258 tokens, and audio is 32 tokens per second. With metadata, each second of video becomes ~300 tokens, which means a 1M context window can fit slightly less than an hour of video.

To ask questions about time-stamped locations, use the format `MM:SS`, where the first two digits represent minutes and the last two digits represent seconds.

For best results:

*   Use one video per prompt.
*   If using a single video, place the text prompt after the video.

### Upload a video file to the File API

**NOTE**: The File API lets you store up to 20 GB of files per project, with a per-file maximum size of 2 GB. Files are stored for 48 hours. They can be accessed in that period with your API key, but they cannot be downloaded using any API. It is available at no cost in all regions where the Gemini API is available.

The File API accepts video file formats directly. This example uses the short NASA film ["Jupiter's Great Red Spot Shrinks and Grows"](https://www.youtube.com/watch?v=JDi4IdtvDVE0). Credit: Goddard Space Flight Center (GSFC)/David Ladd (2018).

> "Jupiter's Great Red Spot Shrinks and Grows" is in the public domain and does not show identifiable people. ([NASA image and media usage guidelines.](https://www.nasa.gov/nasa-brand-center/images-and-media/))

Start by retrieving the short video:

In [44]:
!wget https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4

--2025-02-25 09:25:12--  https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.163.207, 142.251.167.207, 142.251.16.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.163.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238090979 (227M) [video/mp4]
Saving to: ‘GreatRedSpot.mp4’

GreatRedSpot.mp4    100%[===================>] 227.06M   107MB/s    in 2.1s    

2025-02-25 09:25:14 (107 MB/s) - ‘GreatRedSpot.mp4’ saved [238090979/238090979]



Upload the video to the File API and print the URI.

In [45]:
video_file_name = "GreatRedSpot.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/h3l2vsm2yuib


### Verify file upload and check state

Verify the API has successfully received the files by calling the [`files.get`](https://ai.google.dev/api/rest/v1beta/files/get) method.

**NOTE**: Video files have a `State` field in the File API. When a video is uploaded, it will be in the `PROCESSING` state until it is ready for inference. Only `ACTIVE` files can be used for model inference.

In [46]:
import time

# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

..

### Prompt with a video and text

Once the uploaded video is in the `ACTIVE` state, you can make `GenerateContent` requests that specify the File API URI for that video. Select the generative model and provide it with the uploaded video and a text prompt.

In [47]:
# Create the prompt.
prompt = "Summarize this video. Then create a quiz with answer key based on the information in the video."

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([video_file, prompt],
                                  request_options={"timeout": 600})

# Print the response, rendering any Markdown
Markdown(response.text)

Making LLM inference request...


Here’s a summary of the video and a brief quiz to test your comprehension.

This NASA video explains the history and evolution of Jupiter’s Great Red Spot. Jupiter is the largest and oldest planet in the solar system, formed from the same elements as a star but did not become massive enough to ignite.  It’s made of gases and liquids that churn together creating patterns of multicolored cloud bands and the Great Red Spot. The Great Red Spot is a massive storm with no land mass to slow it down, raging for centuries. However, recent studies have shown that the storm is shrinking and changing shape. Scientists, analyzing data from NASA missions like Voyager, Hubble, and Juno, found that the Great Red Spot isn't spinning faster as expected. Instead, it’s getting taller. They compare this to working with pottery on a spinning wheel: as the wide clay spins, the forces push it upward, making it taller while appearing smaller and rounder from above. Though it once fit three Earths, now it only fits a little more than one. Through ongoing research, scientists hope to unravel more secrets of the mysterious Great Red Spot.

Quiz:
1. What type of celestial body is Jupiter?
2. What is the Great Red Spot?
3. Is the Great Red Spot getting bigger or smaller?
4. What is causing the Great Red Spot to change shape?
5. What analogy is used to explain the Great Red Spot’s change in shape?
6. Name one of the NASA missions that gathered data on the Great Red Spot.

Answer Key:
1. A gas giant planet.
2. A massive storm.
3. Smaller.
4. The forces from the spinning of the planet are pushing the storm upwards.
5. Working with pottery on a spinning wheel.
6. Voyager, Hubble, or Juno.

Hopefully, this was helpful!

### Refer to timestamps in the content

You can use timestamps of the form `MM:SS` to refer to specific moments in the video.

In [ ]:
# Create the prompt.
prompt = "What are the examples given at 01:05 and 01:19 supposed to show us?"

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([prompt, video_file],
                                  request_options={"timeout": 600})
Markdown(response.text)

### Transcribe video and provide visual descriptions

The Gemini models can transcribe and provide visual descriptions of video content
by processing both the audio track and visual frames.
For visual descriptions, the model samples the video at a rate of **1 frame
per second**. This sampling rate may affect the level of detail in the
descriptions, particularly for videos with rapidly changing visuals.

In [ ]:
# Create the prompt.
prompt = "Transcribe the audio from this video, giving timestamps for salient events in the video. Also provide visual descriptions."

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([video_file, prompt],
                                  request_options={"timeout": 600})
Markdown(response.text)

Making LLM inference request...


## List files

You can list all uploaded files and their URIs using `files.list_files()`.

In [ ]:
# List all files
for file in genai.list_files():
    print(f"{file.display_name}, URI: {file.uri}")

## Delete files

Files are automatically deleted after 2 days. You can also manually delete them using `files.delete()`.

In [ ]:
genai.delete_file(video_file.name)
print(f'Deleted file {video_file.uri}')